- 추가적으로 다우존스 이외에 kodex 예측
- 첫 시도는 결과가 좋지는 못함

In [1]:
import yfinance as yf
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import itertools
from itertools import combinations

In [845]:
# http://www.wins.or.kr/DataPool/Board/4xxxx/455xx/45587/JKDISS028-02-05.pdf
# 위 링크의 논문을 보고 여러 검색을 통해 코드를 작성

kodex = yf.download('069500.KS',start = start_date, end = end_date)
kodex = kodex.reset_index()[["Date","Close"]]

# https://wikidocs.net/163550
kodex['변화량'] = kodex['Close'] -kodex['Close'].shift(1)
kodex['상승폭'] = np.where(kodex['변화량']>=0, kodex['변화량'], 0)
kodex['하락폭'] = np.where(kodex['변화량'] <0, kodex['변화량'].abs(), 0)

# welles moving average
kodex['AU'] = kodex['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
kodex['AD'] = kodex['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
#df['RS'] = df['AU'] / df['AD']
#df['RSI'] = 100 - (100 / (1 + df['RS']))
kodex['RSI'] = kodex['AU'] / (kodex['AU'] + kodex['AD']) * 100

#https://youngwonhan-family.tistory.com/entry/1-Python-%EC%A3%BC%EC%8B%9D-%EC%9D%BC%EA%B0%84-%EB%B3%80%EB%8F%99%EB%A5%A0-%EA%B3%84%EC%82%B0-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-vs-%EC%95%A0%ED%94%8C-%EC%8B%9C%EA%B0%81%ED%99%94-%EB%B9%84%EA%B5%90
kodex['dpc'] = (kodex.Close - kodex.Close.shift(1)) / kodex.Close.shift(1) * 100
kodex['cs'] = kodex.dpc.cumsum()

#https://wikidocs.net/163553
kodex["Expo"] = kodex['Close'].ewm(span=9, adjust=False).mean()

#https://mkjjo.github.io/finance/2019/07/21/momentom.html
from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

kodex["momentum"] = kodex["Close"].rolling(90).apply(momentum, raw=False)
kodex["diff"] = kodex.Close - kodex.Close.shift(1)
kodex["diff"] = kodex["diff"].apply(lambda x: 1 if x>0 else -1)
kodex = kodex.dropna()


kodex[kodex.columns.difference(['diff'])] = kodex[kodex.columns.difference(['diff'])].shift(1)
kodex = kodex.dropna()

kodex = kodex[['RSI', 'dpc',"cs","Expo","momentum", "diff"]]
x = kodex[kodex.columns.difference(["diff"])]
y = kodex[["diff"]]

x.columns = kodex.columns.difference(["diff"])
y.columns = ["diff"]
    
x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

model = CatBoostClassifier()
model.fit(x.train, y.train)
preds_class = model.predict(x.test)

sum(y.test.values == preds_class)/len(y.test)

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_3717922/487362825.py:47: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_3717922/487362825.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


Learning rate set to 0.013493
0:	learn: 0.6928698	total: 2.93ms	remaining: 2.93s
1:	learn: 0.6924758	total: 5.56ms	remaining: 2.78s
2:	learn: 0.6922024	total: 8.17ms	remaining: 2.71s
3:	learn: 0.6920136	total: 10.6ms	remaining: 2.64s
4:	learn: 0.6916901	total: 12.7ms	remaining: 2.52s
5:	learn: 0.6913425	total: 14.7ms	remaining: 2.44s
6:	learn: 0.6911295	total: 16.7ms	remaining: 2.37s
7:	learn: 0.6907852	total: 18.7ms	remaining: 2.32s
8:	learn: 0.6903014	total: 20.5ms	remaining: 2.26s
9:	learn: 0.6900179	total: 22.1ms	remaining: 2.19s
10:	learn: 0.6896698	total: 23.6ms	remaining: 2.12s
11:	learn: 0.6893891	total: 25.1ms	remaining: 2.07s
12:	learn: 0.6890628	total: 26.6ms	remaining: 2.02s
13:	learn: 0.6886562	total: 28.2ms	remaining: 1.98s
14:	learn: 0.6884063	total: 29.7ms	remaining: 1.95s
15:	learn: 0.6881596	total: 31.1ms	remaining: 1.91s
16:	learn: 0.6878289	total: 32.4ms	remaining: 1.87s
17:	learn: 0.6875693	total: 33.8ms	remaining: 1.84s
18:	learn: 0.6872848	total: 35.1ms	remaining

array([0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.47707559,
       0.52292441, 0.47707559, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292441,
       0.47707559, 0.47707559, 0.47707559, 0.47707559, 0.52292441,
       0.52292441, 0.47707559, 0.52292441, 0.52292441, 0.52292441,
       0.47707559, 0.47707559, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.47707559, 0.47707559, 0.52292441, 0.47707559,
       0.52292441, 0.47707559, 0.52292441, 0.47707559, 0.52292441,
       0.47707559, 0.47707559, 0.52292441, 0.47707559, 0.52292441,
       0.52292441, 0.52292441, 0.47707559, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292441,
       0.52292441, 0.52292441, 0.52292441, 0.52292441, 0.52292

In [846]:
sum(sum(y.test.values.tolist(), []) == preds_class)/len(y.test)

0.506815365551425

In [895]:
# 위 코드를 보니 평활을 하려면 오늘 분석을 한다면 내일의 값까지 있어야하는 경우가 있다.
# 그러므로 평활을 하는 코드는 삭제

kodex = yf.download('069500.KS',start = start_date, end = end_date)
kodex = kodex.reset_index()[["Date","Close"]]
# http://www.wins.or.kr/DataPool/Board/4xxxx/455xx/45587/JKDISS028-02-05.pdf
# https://wikidocs.net/163550
kodex['변화량'] = kodex['Close'] -kodex['Close'].shift(1)
kodex['상승폭'] = np.where(kodex['변화량']>=0, kodex['변화량'], 0)
kodex['하락폭'] = np.where(kodex['변화량'] <0, kodex['변화량'].abs(), 0)

#https://youngwonhan-family.tistory.com/entry/1-Python-%EC%A3%BC%EC%8B%9D-%EC%9D%BC%EA%B0%84-%EB%B3%80%EB%8F%99%EB%A5%A0-%EA%B3%84%EC%82%B0-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-vs-%EC%95%A0%ED%94%8C-%EC%8B%9C%EA%B0%81%ED%99%94-%EB%B9%84%EA%B5%90
kodex['dpc'] = (kodex.Close - kodex.Close.shift(1)) / kodex.Close.shift(1) * 100
kodex['cs'] = kodex.dpc.cumsum()

#https://mkjjo.github.io/finance/2019/07/21/momentom.html
from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

kodex["momentum"] = kodex["Close"].rolling(90).apply(momentum, raw=False)
kodex["diff"] = kodex.Close - kodex.Close.shift(1)
kodex["diff"] = kodex["diff"].apply(lambda x: 1 if x>0 else -1)
kodex = kodex.dropna()

kodex[kodex.columns.difference(['diff'])] = kodex[kodex.columns.difference(['diff'])].shift(1)
kodex = kodex.dropna()


kodex = kodex[['dpc',"cs","momentum", "diff"]]
x = kodex[kodex.columns.difference(["diff"])]
y = kodex[["diff"]]

x.columns = kodex.columns.difference(["diff"])
y.columns = ["diff"]
    
x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

model = CatBoostClassifier()
model.fit(x.train, y.train)
preds_class = model.predict(x.test)


[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_3717922/415457195.py:37: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_3717922/415457195.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


Learning rate set to 0.013493
0:	learn: 0.6929458	total: 2.81ms	remaining: 2.81s
1:	learn: 0.6926846	total: 5.49ms	remaining: 2.74s
2:	learn: 0.6924179	total: 8.1ms	remaining: 2.69s
3:	learn: 0.6921061	total: 10.2ms	remaining: 2.55s
4:	learn: 0.6920171	total: 12ms	remaining: 2.39s
5:	learn: 0.6917589	total: 14.1ms	remaining: 2.34s
6:	learn: 0.6915436	total: 16.1ms	remaining: 2.29s
7:	learn: 0.6912839	total: 18.1ms	remaining: 2.25s
8:	learn: 0.6910379	total: 19.8ms	remaining: 2.18s
9:	learn: 0.6907331	total: 21.4ms	remaining: 2.12s
10:	learn: 0.6905633	total: 22.9ms	remaining: 2.06s
11:	learn: 0.6903863	total: 24.4ms	remaining: 2.01s
12:	learn: 0.6901366	total: 25.9ms	remaining: 1.97s
13:	learn: 0.6899564	total: 27.5ms	remaining: 1.93s
14:	learn: 0.6897645	total: 28.9ms	remaining: 1.9s
15:	learn: 0.6895369	total: 30.1ms	remaining: 1.85s
16:	learn: 0.6893427	total: 31.4ms	remaining: 1.81s
17:	learn: 0.6892202	total: 32.6ms	remaining: 1.78s
18:	learn: 0.6890008	total: 33.8ms	remaining: 1.

In [896]:
sum(sum(y.test.values.tolist(), []) == preds_class)/len(y.test)

0.48203221809169766

- 논문을 이용한 결과는 실패

- 다른 논문에서 window size를 이용해 분석을 해 좋은 결과를 얻음 -> window size를 고려

In [3]:

start_date = "2011-12-30" # 2012-01-01 이후 첫번째 데이터가 Nan인 경우 선형 보간법을 사용하기 위해 앞의 날짜부터 가져옴
end_date = "2022-04-30"
stock_df = yf.download(['069500.KS', '^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS", "KRW=X", "GC=F", "CL=F"],start = start_date, end = end_date).Close
stock_df = stock_df.reset_index()
stock_df = stock_df.dropna(subset = ["069500.KS"])
stock_df[stock_df.columns.difference(['Date'])] = stock_df[stock_df.columns.difference(['Date'])].interpolate(method='linear',axis=0)
stock_df = stock_df.dropna()

difference = 1
stock_df = stock_df[stock_df.columns.difference(["Date"])]
stock_df = stock_df.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
stock_df = stock_df.dropna()
stock_df = stock_df.apply(lambda y: y.apply(lambda x: 1 if x >= 0.003 else (-1 if x<=-0.003 else 0)))
stock_df[stock_df.columns.difference(['069500.KS'])] = stock_df[stock_df.columns.difference(['069500.KS'])].shift(difference)
stock_df[stock_df.columns.difference(['069500.KS', '^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS"])] = stock_df[stock_df.columns.difference(['069500.KS', '^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS"])].shift(difference)
stock_df = stock_df.dropna()

x = stock_df[stock_df.columns.difference(["069500.KS"])]
y = stock_df[["069500.KS"]]

x.columns = stock_df.columns.difference(["069500.KS"])
y.columns = ["069500.KS"]

x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

x_train_window = x.train
x_test_window = x.test
y_train_window = y.train
y_test_window = y.test

window_size = 4
for i in range(window_size):
    print(i)
    x_train_window = pd.concat([x_train_window, x.train.shift(i+1)], axis = 1)
    x_test_window = pd.concat([x_test_window, x.test.shift(i+1)], axis = 1)
    
x_train_window = x_train_window.dropna()
x_test_window = x_test_window.dropna()

y_train_window = y_train_window.iloc[window_size:len(y_train_window)]
y_test_window = y_test_window.iloc[window_size:len(y_test_window)]

x_train_window.columns = list(range(0,x_train_window.shape[1]))
x_test_window.columns = list(range(0,x_test_window.shape[1]))


model=CatBoostClassifier()
model.fit(x_train_window, y_train_window)
preds_class = model.predict(x_test_window)
#    pred.append(sum(y.test["069500.KS"].values == preds_class.reshape(len(x.test)))/len(y.test))

[*********************100%***********************]  9 of 9 completed
0
1
2
3
Learning rate set to 0.081453
0:	learn: 1.0906090	total: 51.4ms	remaining: 51.4s
1:	learn: 1.0825505	total: 54.1ms	remaining: 27s
2:	learn: 1.0729429	total: 57.1ms	remaining: 19s
3:	learn: 1.0653012	total: 59.7ms	remaining: 14.9s
4:	learn: 1.0577868	total: 61.8ms	remaining: 12.3s
5:	learn: 1.0507656	total: 63.9ms	remaining: 10.6s
6:	learn: 1.0440150	total: 65.9ms	remaining: 9.35s
7:	learn: 1.0386512	total: 67.9ms	remaining: 8.42s
8:	learn: 1.0355738	total: 69.6ms	remaining: 7.67s
9:	learn: 1.0300732	total: 71.3ms	remaining: 7.06s
10:	learn: 1.0247276	total: 72.9ms	remaining: 6.55s
11:	learn: 1.0197674	total: 74.4ms	remaining: 6.12s
12:	learn: 1.0151246	total: 75.8ms	remaining: 5.76s
13:	learn: 1.0108800	total: 78.3ms	remaining: 5.51s
14:	learn: 1.0053922	total: 80.3ms	remaining: 5.27s
15:	learn: 1.0011767	total: 82ms	remaining: 5.04s
16:	learn: 0.9963727	total: 83.5ms	remaining: 4.83s
17:	learn: 0.9926149	tota

/tmp/ipykernel_2823912/2400393722.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2823912/2400393722.py:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


24:	learn: 0.9636240	total: 95.5ms	remaining: 3.72s
25:	learn: 0.9598367	total: 96.6ms	remaining: 3.62s
26:	learn: 0.9566511	total: 97.8ms	remaining: 3.52s
27:	learn: 0.9531470	total: 98.9ms	remaining: 3.43s
28:	learn: 0.9494522	total: 100ms	remaining: 3.35s
29:	learn: 0.9457040	total: 101ms	remaining: 3.27s
30:	learn: 0.9417921	total: 102ms	remaining: 3.19s
31:	learn: 0.9382387	total: 103ms	remaining: 3.12s
32:	learn: 0.9357319	total: 104ms	remaining: 3.05s
33:	learn: 0.9326408	total: 105ms	remaining: 2.99s
34:	learn: 0.9294654	total: 106ms	remaining: 2.93s
35:	learn: 0.9264678	total: 107ms	remaining: 2.87s
36:	learn: 0.9228256	total: 108ms	remaining: 2.82s
37:	learn: 0.9195720	total: 109ms	remaining: 2.77s
38:	learn: 0.9162317	total: 110ms	remaining: 2.71s
39:	learn: 0.9132427	total: 111ms	remaining: 2.67s
40:	learn: 0.9099507	total: 112ms	remaining: 2.62s
41:	learn: 0.9071902	total: 113ms	remaining: 2.58s
42:	learn: 0.9040819	total: 114ms	remaining: 2.54s
43:	learn: 0.9003542	total:

In [ ]:
sum(y_test_window["069500.KS"].values == preds_class.reshape(len(x_test_window)))/len(x_test_window)

0.4101326899879373

-> 실패
- Stock Market Forecasting Using Machine Learning Algorithms, 2012
- 위 논문의 내용을 참고해 임의로 변수를 선택하고 다양한 변수 조합으로 분석 후 정확도 기준 최고의 변수 조합을 뽑아냄

In [ ]:
start_date = "2011-12-30" # 2012-01-01 이후 첫번째 데이터가 Nan인 경우 선형 보간법을 사용하기 위해 앞의 날짜부터 가져옴
end_date = "2022-04-30"
stock = ['^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS", "KRW=X", "GC=F", "CL=F"]
stock_vari = []
for i in range(len(stock)+1):
    for vari in combinations(stock, i):
        stock_vari.append(list(vari))

stock_vari = [['069500.KS']+stock_vari[i] for i in range(len(stock_vari))]

pred = []
for i in stock_vari[1:]:
    stock_df = yf.download(i,start = start_date, end = end_date).Close
    stock_df = stock_df.reset_index()
    stock_df = stock_df.dropna(subset = ["069500.KS"])
    stock_df[stock_df.columns.difference(['Date'])] = stock_df[stock_df.columns.difference(['Date'])].interpolate(method='linear',axis=0)
    stock_df = stock_df.dropna()

    difference = 1
    stock_df = stock_df[stock_df.columns.difference(["Date"])]
    stock_df = stock_df.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)
    stock_df = stock_df.dropna()

    # 0.003 기준 : Stock Market Trend Forecasting Based on Multiple Textual Features: A Deep Learning Method
    stock_df = stock_df.apply(lambda y: y.apply(lambda x: 1 if x > 0.003 else (-1 if x<=-0.003 else 0)))
    #stock_df["Close"] = stock_df["Close"].apply(lambda y: 1 if y >= 0.003 else (-1 if y<=-0.003 else 0))
    '''
    stock_df = stock_df.apply(lambda x: x/abs(x), axis = 0)
    stock_df = stock_df.dropna()
    stock_df = stock_df.reset_index(drop=True)
    '''

    stock_df[stock_df.columns.difference(['069500.KS'])] = stock_df[stock_df.columns.difference(['069500.KS'])].shift(difference)
    stock_df[stock_df.columns.difference(['069500.KS', '^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS"])] = stock_df[stock_df.columns.difference(['069500.KS', '^DJI',"^KS11", "^KS200", "^KQ11", "005930.KS"])].shift(difference)
    stock_df = stock_df.dropna()

    x = stock_df[stock_df.columns.difference(["069500.KS"])]
    y = stock_df[["069500.KS"]]

    x.columns = stock_df.columns.difference(["069500.KS"])
    y.columns = ["069500.KS"]

    x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
    y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

    model = CatBoostClassifier(verbose = 0)
    model.fit(x.train, y.train)
    preds_class = model.predict(x.test)
    pred.append(sum(y.test["069500.KS"].values == preds_class.reshape(len(x.test)))/len(y.test))

In [27]:
# 가장 좋은 변수 조합으로 분석
start_date = "2011-12-30" # 2012-01-01 이후 첫번째 데이터가 Nan인 경우 선형 보간법을 사용하기 위해 앞의 날짜부터 가져옴
end_date = "2022-04-30"
i = ['069500.KS', '^DJI', '^KS11', '005930.KS']
stock_df = yf.download(i,start = start_date, end = end_date).Close
stock_df = stock_df.reset_index()

stock_df = stock_df.dropna(subset = ["069500.KS"])
stock_df[stock_df.columns.difference(['Date'])] = stock_df[stock_df.columns.difference(['Date'])].interpolate(method='linear',axis=0)
stock_df = stock_df.drop(["Date"], axis = 1)

difference = 1
stock_df = stock_df[stock_df.columns.difference(["Date"])]
stock_df = stock_df.apply(lambda x: (x - x.shift(difference))/x.shift(difference), axis = 0)


# 0.003 기준 : Stock Market Trend Forecasting Based on Multiple Textual Features: A Deep Learning Method

'''
stock_df = stock_df.apply(lambda x: x/abs(x), axis = 0)
stock_df = stock_df.dropna()
stock_df = stock_df.reset_index(drop=True)
'''

stock_df[stock_df.columns.difference(['069500.KS'])] = stock_df[stock_df.columns.difference(['069500.KS'])].shift(difference)
stock_df[stock_df.columns.difference(['069500.KS',"^KS11", "^KS200", "^KQ11", "005930.KS"])] = stock_df[stock_df.columns.difference(['069500.KS',"^KS11", "^KS200", "^KQ11", "005930.KS"])].shift(difference)
stock_df = stock_df.apply(lambda y: y.apply(lambda x: 1 if x > 0.003 else (-1 if x<=-0.003 else 0)))
#stock_df["Close"] = stock_df["Close"].apply(lambda y: 1 if y >= 0.003 else (-1 if y<=-0.003 else 0))
stock_df = stock_df.iloc[2:len(stock_df)]
stock_df = stock_df.fillna(0)

x = stock_df[stock_df.columns.difference(["069500.KS"])]
y = stock_df[["069500.KS"]]

x.columns = stock_df.columns.difference(["069500.KS"])
y.columns = ["069500.KS"]

x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)

model = CatBoostClassifier(verbose = 0)
model.fit(x.train, y.train)
preds_class = model.predict(x.test)
sum(y.test["069500.KS"].values == preds_class.reshape(len(x.test)))/len(y.test)

[*********************100%***********************]  4 of 4 completed


/tmp/ipykernel_3677864/2039928161.py:37: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_3677864/2039928161.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


0.3069828722002635

-> 처참한 결과 => 수정이 필요

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, GridSearchCV
from xgboost import XGBClassifier

In [5]:
# 결과가 그나마 나앗던 변수 조합으로 XGBoost 실행
x = x[['^DJI', '^KS11', '005930.KS']]
model=XGBClassifier()
param_grid={'booster' :['gbtree'],
                 'max_depth':[3,5,7,9,11],
                 'min_child_weight':[3,5,7],
                 'gamma':[0,2,4,6,8,10],
                 'nthread':[50],
                 'colsample_bytree':[0.3,0.5,0.7, 0.9],
                 'n_estimators':[50, 100, 150],
                 'objective':['binary:logistic'],
                 'random_state':[42]}

le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))
x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)
cv=KFold(n_splits=6, shuffle = True, random_state=42)
gcv=GridSearchCV(model, param_grid=param_grid, cv=cv, scoring='accuracy')
gcv.fit(x.train, y.train)

print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력
print('best score', gcv.best_score_)

/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/tmp/ipykernel_2823912/673027137.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.train, x.test = train_test_split(x, train_size = 0.7, shuffle = False)
/tmp/ipykernel_2823912/673027137.py:17: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  y.train, y.test = train_test_split(y, train_size = 0.7, shuffle = False)


final params {'booster': 'gbtree', 'colsample_bytree': 0.3, 'gamma': 4, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'nthread': 50, 'objective': 'binary:logistic', 'random_state': 42}
best score 0.4533783004727314
